DBSCAN
===============
_Density-based spatial clustering of applications with noise_
***

TODO: Explanantion of DBSCAN

Clustering Of _Waking-up_ and _Sleep_ timings
=================
*****************
The following algorithm below will cluster sleep periods using the [DBSCAN algorithm](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) from the [scikit-learn library](http://scikit-learn.org/)

Input data requirements
---------------------
****
The data input is expected to follow the following format:

<table>
<th>TODO</th>
</table>

In [7]:
%matplotlib inline
###########################################IMPRORTS#########################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pi
from sklearn.cluster import DBSCAN, KMeans
import eps_min_pts as eps_And_MinPts
import helper_adl as h
from matplotlib.backends.backend_pdf import PdfPages
###########################################IMPRORTS#########################################


# To return value in mins / total mins in one day
def to_mins(x):
    x = pd.Timestamp(x)
    year = x.year
    month =  x.month
    day = x.day
    return (x.value - pd.Timestamp(str(year)+'-'+str(month)+'-'+str(day)).value) / (60 * (10**9))

# Helper method to convert values to radian
def convert_to_radian(x):
    return ((x / (24*60)) * 2 * pi)

# Returns a distance matrix (a numpy array)
def get_x_from_df(series):
    
    # Vectorizing to_mins and to_radian functions
    tmin = np.vectorize(to_mins)
    trad = np.vectorize(convert_to_radian)

    # Converting series of timestamp -> minutes / total minuites in a day -> radian
    input_rad = trad(tmin(series))

    # Convert time to rad points   
    X = input_rad[None,:] - input_rad[:,None]

    # Assign 'shortest distance to each point
    X[((X > pi) & (X <= (2*pi)))] = X[((X > pi) & (X <= (2*pi)))] -(2*pi)
    X[((X > (-2*pi)) & (X <= (-1*pi)))] = X[((X > (-2*pi)) & (X <= (-1*pi)))] + (2*pi) 
    X = abs(X)

    return X,input_rad

# Returns output from sklearn DBSCAN method
def dbscan(eps, min_pts, X, metric='precomputed'):
    db = DBSCAN(eps, min_pts, metric)
    db.fit(X)
    return db.labels_, db.components_, db.core_sample_indices_

def configure_polar_plot(axes):
    xticklabels = [str(x) for x in range (0,24)]
    axes.set_xticks(np.linspace(0,23,24,endpoint=True, dtype=np.int32) / 24 * (2 * np.pi))
    axes.set_xticklabels(xticklabels)
    axes.set_theta_zero_location('N')
    axes.set_theta_direction(-1)
    axes.set_ylim([0,1])
    axes.set_yticks([0.8,0.6])
    axes.set_yticklabels(['Sleep-Start', 'Sleep-End'])
    
    return axes

def plot_clusters(labels, rad_tseries, axes, ring):
    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    for k,col in zip(unique_labels, colors):
        marker = '.'
        ms = 30.0
        if k == -1:
            col='k'
            marker = 'x'
            ms = 8.0

        indices_of_k, = np.where(labels == k)
        data = rad_tseries.take(indices_of_k)
        axes.plot(data, [ring for x in data], color=col, marker=marker, linestyle='none', ms=ms, mec='k')
        

In [23]:
# Code
# Start timer:
time_now = time.time()

### Globals ###
file_dir = 'sleep'
# Perform clustering on files 1 to 10
# indexes = [str(x) for x in range(1,10)]
index = '9'

file_name = ''.join([file_dir,'/','sleep-aggregate_2016-01_S001.csv'])
# print file name
print('File name: ', file_name)

# Get dataframe containing readings from sensor reading, exclude 
df = pd.read_csv(file_name, delimiter=',', usecols=[x for x in range(1,7)],parse_dates=[1])
column_names = list(df.columns.values)
# print(column_names)

# X is a distance matrix.
# Set 'X1' as sleep_start timings
X1,X1_rad_series = get_x_from_df(df['sleep_start'])

# Set 'X2' as sleep_end timings
X2,X2_rad_series = get_x_from_df(df['sleep_end'])


# Arbitrary eps and min_pts value:
eps_X1, min_pts_X1 = eps_And_MinPts.knee_calculate_eps_minPts(X1)
print('eps: ', eps_X1 , ' minPts: ' , min_pts_X1)
eps_X2, min_pts_X2 = eps_And_MinPts.knee_calculate_eps_minPts(X2)
print('eps: ', eps_X2 , ' minPts: ' , min_pts_X2)
# eps = 0.31
# min_pts = 29

X1_label, X1_components, X1_csi = dbscan(eps_X1, min_pts_X1, X1)
X2_label, X2_components, X2_csi = dbscan(eps_X2, min_pts_X2, X2)

print(X1_label)
print(X1_components)
print(X1_csi)
# Sanity check here:
# print('Sanity check.......')
# print('Checking.........................')
# print('start sleep time cluster: ')

# - 1 if -1 exist in labels because -1 is used to denote noise
X1_no_clusters = len(set(X1_label)) - (1 if -1 in X1_label else 0)
print('Number of clusters for start sleep time: ', X1_no_clusters )
# print('end sleep time cluster: ')
X2_no_clusters = len(set(X2_label)) - (1 if -1 in X2_label else 0)
print('Number of clusters for end sleep time: ', X2_no_clusters )
cluster_list = []

# XX is a df containing X1 and X2, used for Kmeans clustering of 2 clusters
XX = pd.DataFrame();

print('len of X1_rad_series, ', len(X1_rad_series))
print('len of X2_rad_series, ', len(X2_rad_series))
print('len of np.where for X1_labels ', len(X1_label ))
print('len of np.where for X2_labels ', len(X2_label ))

indices_of_X1_without_noise, = np.where(X1_label != -1)
print(indices_of_X1_without_noise)
X1_no_noise = X1_rad_series[indices_of_X1_without_noise]
indices_of_X2_without_noise, = np.where(X2_label != -1)
print(indices_of_X2_without_noise)
X2_no_noise = X2_rad_series[indices_of_X2_without_noise]

# XX['X1'] = X1_no_noise
# XX['X2'] = X2_no_noise

print('What is XX? :', XX)

km = KMeans(n_clusters=2, init='k-means++', precompute_distances='auto', n_jobs=-1)
km.fit(XX.as_matrix())
XX_cluster_centers = km.cluster_centers_ 
XX_labels = km.labels_
XX_inertia = km.inertia_

print('KMeans')
print('--------------------------')
# print(XX_labels)
# print(XX_cluster_centers)
# print(XX_inertia)

# Remove noise first


km_clusters = set(XX_labels)
km_dict = {}
for k in km_clusters:
    indices_of_k = np.where(XX_labels == k)
    X1_min = np.min(X1_rad_series[indices_of_k])
    X2_max = np.max(X2_rad_series[indices_of_k])
    km_dict[k] = (X1_min, X2_max)
    
print(km_dict)


# EXTRACTION OF CLUSTERS
###########################################
# cluster_dict_ = h.extract_clusters(X1_label, X1_rad_series)
# cluster_dict_keys_ = cluster_dict_.keys()
# for i in cluster_dict_keys_:
#     cluster = cluster_dict_[i]
#     print(cluster)
#     cluster_list.append(np.asscalar(i))
#     print('Cluster : ', i)
#     print('SD : ', np.std(cluster))
#     print('VAR : ', np.var(cluster))
#     print('centroid-val : ', np.median(cluster))


# print(cluster_list)
##################PLOTTING####################
# TODO: CLUSTER
fig = plt.figure(figsize=(6,6))
ax1 = configure_polar_plot(fig.add_subplot(111, projection='polar'))
ax1.set_title(file_name)
# ax1.plot(X1_rad_series, [1 for x in X1_rad_series], 'mo')
# ax1.plot(X2_rad_series, [0.8 for x in X2_rad_series], 'y*')
plot_clusters(X1_label, X1_rad_series, ax1, 0.8)
plot_clusters(X2_label, X2_rad_series, ax1, 0.6)

km_clust_0_min,km_clust_0_max = km_dict[0]
km_clust_1_min,km_clust_1_max = km_dict[1]
ax1.bar(km_clust_0_min,1, width=km_clust_0_max - km_clust_0_min, bottom=0.0)
ax1.bar(km_clust_1_min ,1, width=km_clust_1_max - km_clust_1_min, bottom=0.0)
# ax1.plot(km_clust_0_max,1, marker = '|', ms=380.0)
# ax1.plot(km_clust_1_min,1, marker = '|', ms=380.0)
# ax1.plot(km_clust_1_max,1, marker = '|', ms=380.0)
# ax1.plot(km_dict[1])
plt.show()

print("Elasped Time: ", round(time.time() - time_now, 3), "seconds")
    

File name:  sleep/sleep-aggregate_2016-01_S001.csv
[ 0.          2.11533905  1.0608693  ...,  2.54832615  1.46556752  0.        ]
eps:  0.314726497366  minPts:  14.0
[ 0.          0.74918258  1.92473455 ...,  2.42193523  1.89622751  0.        ]
eps:  0.314595597672  minPts:  14.5
[-1  1  0 -1  1  1 -1  2 -1  1  2  0 -1 -1  0 -1  1  0  0  1  2 -1 -1  1  2
 -1  0 -1  2 -1  0 -1  0 -1 -1 -1  1  0  0  2  0  1  2  0  1  2  0  1 -1  2
  0  2  0  0 -1 -1 -1  0 -1 -1  0  1  2 -1  1  2  0  0 -1  2  0  2  0 -1  1
  2 -1 -1 -1  1  2  0 -1 -1  1  0 -1  1  2 -1  0  0  2  0  1 -1  0]
[[ 1.0608693   3.10697696  0.         ...,  2.79339947  1.2204942
   0.24507332]
 [ 1.51618207  0.59915699  2.57705136 ...,  0.21634811  2.48563974
   2.33197805]
 [ 3.05985307  1.10799319  1.99898377 ...,  1.49080207  0.77848957
   2.24405709]
 ..., 
 [ 1.15671696  3.01112929  0.09584766 ...,  2.88924713  1.12464654
   0.34092098]
 [ 1.73253017  0.38280888  2.79339947 ...,  0.          2.26929164
   2.54832615]
 [ 0.81

ValueError: Length of values does not match length of index